In [1]:
using Pkg
Pkg.activate(@__DIR__)
using RigidBodyDynamics, RigidBodyDynamics.Contact, RigidBodyDynamics.OdeIntegrators
using EnhancedGJK
using StaticArrays
using CoordinateTransformations
using Rotations
using MeshCatMechanisms, MeshCat
using GeometryTypes
using RigidBodyDynamics: transform, frame_definition
# using Polyhedra

┌ Info: Recompiling stale cache file /home/twan/.julia/compiled/v1.1/RigidBodyDynamics/WeevQ.ji for RigidBodyDynamics [366cf18f-59d5-5db9-a4de-86a9f6786172]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/twan/.julia/compiled/v1.1/MeshCatMechanisms/dGmNl.ji for MeshCatMechanisms [6ad125db-dd91-5488-b820-c1df6aab299d]
└ @ Base loading.jl:1184


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/29d4a42ceb6d03d53ba5feb8981c0eb5bceb5139-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/29d4a42ceb6d03d53ba5feb8981c0eb5bceb5139-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
# struct AbstractTypeDict end

# @inline function Base.getindex(d::AbstractTypeDict, ::Type{T}) where {T}
#     ReturnType = predict_type(d, T)
#     id = objectid(T)
#     ids = keys(d)
#     vals = values(d)
#     @inbounds for i in eachindex(ids)
#         if ids[i] === id
#             return vals[i]::ReturnType
#         end
#     end
#     val = make_value(d, T)::ReturnType
#     push!(ids, id)
#     push!(vals, val)
#     val::ReturnType
# end

In [3]:
# struct SimilarTypeDict{V} <: AbstractTypeDict
#     keys::Vector{UInt}
#     values::Vector{V}
# end

# function SimilarTypeDict{V}(base_value, ::Type{T}=eltype(base_value))
# end

# Base.keys(d::SimilarTypeDict) = d.keys
# Base.values(d::SimilarTypeDict) = d.values
# predict_type(::SimilarTypeDict{V}, ::Type{T}) where {V, T} = V{T}


In [4]:
# struct SimilarTypeDict{V, S, P}
#     keys::Vector{UInt}
#     values::Vector{V}
#     similar::S
#     predict_type::P
# end

# function SimilarTypeDict{V}(base_value, ::Type{T}=eltype(base_value);
#         similar::S=similar, predict_type::P=T -> V{T}) where {V, S, P, T}
#     key = objectid(T)
#     return SimilarTypeDict{V, S, P}([key], [base_value], similar, predict_type)
# end

In [5]:
# using BenchmarkTools
# d = SimilarTypeDict{Vector}(rand(3));
# @btime $d[Int]
# @btime $d[BigInt]

In [6]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)
world = root_body(mechanism)
worldframe = root_frame(mechanism)
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [7]:
# Base.similar(state::MechanismState, ::Type{T}) where {T} = MechanismState{T}(state.mechanism)
# predict_type()
# d = SimilarTypeDict{MechanismState{X, Float64, X, typeof(state.treejoints)} where {M, C, JointCollection} where X}(state)

In [8]:
# using BenchmarkTools
# @btime $d[Float64]

In [9]:
lower_link = last(bodies(mechanism))
tipframe = CartesianFrame3D("tip")
add_frame!(lower_link, Transform3D(tipframe, default_frame(lower_link), SVector(0.0, 0.0, -2.05)))
tip = Point3D(tipframe, zero(SVector{3}))

Point3D in "tip": [0.0, 0.0, 0.0]

In [10]:
boxframe = CartesianFrame3D("box")
add_frame!(world, Transform3D(boxframe, worldframe, RotY(-3e-1), SVector(2.5, 0, -0.5)))
planeframe = CartesianFrame3D("plane")
add_frame!(world, Transform3D(planeframe, worldframe, RotY(3e-1), SVector(0, 0, -0.5)))

Transform3D from "plane" to "world":
rotation: 0.3 rad about [1.62667e-62, 1.0, 0.0], translation: [0.0, 0.0, -0.5]

In [11]:
widths = Vec(1.5, 1, 0.3)
c1 = HyperRectangle(-widths / 2, widths)
c2 = HalfSpace(SVector(0., 0., 1.), -1.5)

HalfSpace{3,Float64}([0.0, 0.0, 1.0], -1.5)

In [12]:
if !(@isdefined mvis) || mvis.state != state || !any(isopen, mvis.visualizer.core.scope.pool.connections)
    mvis = MechanismVisualizer(state, URDFVisuals(urdf))
#     open(mvis)
end

MechanismVisualizer{MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{Joint{Float64,Revolute{Float64}},1}},1}},Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat, 5)

In [13]:
set_configuration!(state, [-pi / 2, -1.]) # well out of contact
# set_configuration!(state, [-0.61, -1.36]) # in contact
set_configuration!(mvis, configuration(state))
rand_velocity!(state)
x0 = Vector(state);

In [14]:
setelement!(mvis, tipframe)
setelement!(mvis, tip, 0.03, "tip")
boxvis = mvis.visualizer["box"]
setobject!(boxvis, c1)
settransform!(boxvis, AffineMap(frame_definition(world, boxframe)))
closest_point_vis = mvis.visualizer["closest point"]
setobject!(closest_point_vis, Sphere(Point(0., 0, 0), 0.03))

MeshCat Visualizer with path /meshcat/closest point

In [15]:
for i = 1 : 4
    setobject!(mvis.visualizer["simplex_point_$i"], Sphere(Point(0., 0, 0), 0.03))
end

In [16]:
contact_model = ContactModel()
points = CollisionElement[]
push!(points, CollisionElement(lower_link, tipframe, Point(0., 0, 0)))
push!(contact_model, points)
environment = CollisionElement[]
push!(environment, CollisionElement(world, boxframe, c1))
push!(environment, CollisionElement(world, planeframe, c2))
push!(contact_model, environment)
normal_model = hunt_crossley_hertz(; k=50e3, α = 0.2)
tangential_model = ViscoelasticCoulombModel(0.5, 1e3, 1e3)
contact_force_model = SplitContactForceModel(normal_model, tangential_model)
set_contact_force_model!(contact_model, points, environment, contact_force_model);

In [17]:
contact_state = SoftContactState(contact_model)
contact_result = SoftContactResult(mechanism, contact_model)

SoftContactResult{Float64}(…)

In [28]:
s = (state,)

(MechanismState{Float64, Float64, Float64, …}(…),)

In [29]:
@code_llvm OdeIntegrators.has_additional_state(typeof(s))


;  @ /home/twan/.julia/dev/RigidBodyDynamics/src/simulate.jl:18 within `has_additional_state'
define i8 @julia_has_additional_state_15043(%jl_value_t addrspace(10)*) {
top:
  ret i8 0
}


In [21]:
s = RigidBodyDynamics.CombinedState((state, contact_state, state, contact_state));

MethodError: MethodError: no method matching RigidBodyDynamics.CombinedState(::Tuple{MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{Joint{Float64,Revolute{Float64}},1}},1}},SoftContactState{Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Contact.CollidablePair{CollisionElement{Transform3D{Float64},Point{3,Float64}},CollisionElement{Transform3D{Float64},HyperRectangle{3,Float64}},SplitContactForceModel{HuntCrossleyModel{Float64},ViscoelasticCoulombModel{Float64}}},1},Array{RigidBodyDynamics.Contact.CollidablePair{CollisionElement{Transform3D{Float64},Point{3,Float64}},CollisionElement{Transform3D{Float64},HalfSpace{3,Float64}},SplitContactForceModel{HuntCrossleyModel{Float64},ViscoelasticCoulombModel{Float64}}},1}},2},TypeSortedCollections.TypeSortedCollection{Tuple{Array{CollisionCache{Point{3,Float64},HyperRectangle{3,Float64},4,EnhancedGJK.Difference{EnhancedGJK.Tagged{SArray{Tuple{3},Float64,1,3},Nothing

In [ ]:
Od

In [20]:
typeof(s).parameters

svec(Tuple{MechanismState{Float64,Float64,Float64,TypeSortedCollection{Tuple{Array{Joint{Float64,Revolute{Float64}},1}},1}},SoftContactState{Float64,TypeSortedCollection{Tuple{Array{CollidablePair{CollisionElement{Transform3D{Float64},Point{3,Float64}},CollisionElement{Transform3D{Float64},HyperRectangle{3,Float64}},SplitContactForceModel{HuntCrossleyModel{Float64},ViscoelasticCoulombModel{Float64}}},1},Array{CollidablePair{CollisionElement{Transform3D{Float64},Point{3,Float64}},CollisionElement{Transform3D{Float64},HalfSpace{3,Float64}},SplitContactForceModel{HuntCrossleyModel{Float64},ViscoelasticCoulombModel{Float64}}},1}},2},TypeSortedCollection{Tuple{Array{CollisionCache{Point{3,Float64},HyperRectangle{3,Float64},4,Difference{Tagged{SArray{Tuple{3},Float64,1,3},Nothing},Tagged{SArray{Tuple{3},Float64,1,3},Nothing}}},1},Array{Pair{Point{3,Float64},HalfSpace{3,Float64}},1}},2}},MechanismState{Float64,Float64,Float64,TypeSortedCollection{Tuple{Array{Joint{Float64,Revolute{Float64}},1

In [ ]:
@code_native OdeIntegrators.has_configuration_velocity(s)

In [38]:
@code_warntype OdeIntegrators.has_configuration_velocity(typeof(s))

Body::Bool
1 ─     return true


In [96]:
# Contact.contact_dynamics!(contact_result, contact_state, state)
# contact_result.wrenches[BodyID(3)]

In [97]:
# Base.show(io::IO, c::Core.Compiler.Const) = print(io, "Const(", repr(c.val), ")")
# @code_warntype Contact.contact_dynamics!(contact_result, contact_state, state);

In [98]:
# using BenchmarkTools
# @btime Contact.contact_dynamics!($contact_result, $contact_state, $state);

In [99]:
using EnhancedGJK
using GeometryTypes: HyperRectangle, Vec, Point
import StaticArrays: SVector
import CoordinateTransformations: IdentityTransformation, Translation

# widths = Vec(1.5, 1, 0.3)
# c = HyperRectangle(-widths / 2, widths)
c = Point(rand(), rand(), rand())
cache = CollisionCache(c, zero(SVector{3}));
result = gjk!(cache, IdentityTransformation(), Translation(0., 1., 2.))

# result.signed_distance will be > 0 if the objects are not in contact
# and <= 0 if they are in collision.
@show result.signed_distance
closest_point = result.closest_point_in_body.a

using BenchmarkTools
@btime gjk!($cache, IdentityTransformation(), Translation(0., 1., 2.))
using Profile; Profile.clear_malloc_data()
@btime gjk!($cache, IdentityTransformation(), Translation(0., 1., 2.))

result.signed_distance = 1.3494488524296564
  74.566 ns (1 allocation: 496 bytes)
  71.034 ns (1 allocation: 496 bytes)


GJKResult{4,3,Float64}(SArray{Tuple{3},Float64,1,3}[[0.364235, -0.379813, -1.24261], [0.364235, -0.379813, -1.24261], [0.364235, -0.379813, -1.24261], [0.364235, -0.379813, -1.24261]], EnhancedGJK.Difference{SArray{Tuple{3},Float64,1,3},SArray{Tuple{3},Float64,1,3}}([0.364235, 0.620187, 0.757387], [0.0, 0.0, 0.0]), 1.3494488524296564)

In [100]:
using BenchmarkTools
@btime gjk!($cache, IdentityTransformation(), $(Translation(0., 1., 2.)))

  73.655 ns (1 allocation: 496 bytes)


GJKResult{4,3,Float64}(SArray{Tuple{3},Float64,1,3}[[0.364235, -0.379813, -1.24261], [0.364235, -0.379813, -1.24261], [0.364235, -0.379813, -1.24261], [0.364235, -0.379813, -1.24261]], EnhancedGJK.Difference{SArray{Tuple{3},Float64,1,3},SArray{Tuple{3},Float64,1,3}}([0.364235, 0.620187, 0.757387], [0.0, 0.0, 0.0]), 1.3494488524296564)

In [101]:
# cache = CollisionCache(c1, zero(SVector{3}));
# function control(τ, t, state)
#     pt_world = transform(state, tip, world_frame)
#     result = gjk!(cache, IdentityTransformation(), Translation(pt_world.v))
# #     @show result.signed_distance
# #     set_configuration!(mvis, configuration(state))
#     settransform!(closest_point_vis, Translation(result.closest_point_in_body.a))
#     if result.signed_distance < 0
#         sleep(1e-1)
#     end
    
# #     @show result.simplex
#     p = polyhedron(vrep(result.simplex))
#     setobject!(mvis.visualizer[:simplex], Polyhedra.Mesh(p))
#     for i = 1 : 4
#         point = result.simplex[i]
#         settransform!(mvis.visualizer["simplex_point_$i"], Translation(point))
#     end
# #     sleep(1e-3)
    
#     τ .= 0
#     return τ
# end

In [102]:
# using RigidBodySim

In [103]:
# dynamics = Dynamics(mechanism, control);

In [104]:
# gui = GUI(mvis)
# copyto!(state, x0)
# problem = ODEProblem(dynamics, state, (0., 1.0), callback=CallbackSet(CallbackSet(gui), RealtimeRateLimiter(poll_interval=1e-3, max_rate=0.2)))

In [105]:
#open(gui)

In [106]:
# solve(problem, Tsit5(), dt=1e-3);

In [107]:
# ts, qs, vs = simulate(state, 3.0, control, max_realtime_rate=1.);

In [108]:
# setanimation!(mvis, ts, qs)

In [109]:
OdeIntegrators.configuration(x::Tuple{<:MechanismState, <:SoftContactState}) = configuration(x[1])
OdeIntegrators.velocity(x::Tuple{<:MechanismState, <:SoftContactState}) = velocity(x[1])
OdeIntegrators.additional_state(x::Tuple{<:MechanismState, <:SoftContactState}) = x[2].x

OdeIntegrators.set_configuration!(x::Tuple{<:MechanismState, <:SoftContactState}, q) = set_configuration!(x[1], q)
OdeIntegrators.set_velocity!(x::Tuple{<:MechanismState, <:SoftContactState}, v) = set_velocity!(x[1], v)
OdeIntegrators.set_additional_state!(x::Tuple{<:MechanismState, <:SoftContactState}, s) = copyto!(x[2].x, s)

OdeIntegrators.global_coordinates!(x::Tuple{<:MechanismState, <:SoftContactState}, q0, ϕ) = global_coordinates!(x[1], q0, ϕ)
OdeIntegrators.local_coordinates!(ϕ, ϕd, x::Tuple{<:MechanismState, <:SoftContactState}, q0) = local_coordinates!(ϕ, ϕd, x[1], q0)

In [110]:
import RigidBodyDynamics: cache_eltype
import RigidBodyDynamics.CustomCollections: ConstVector
function simulate_hack(state, final_time; Δt = 1e-4) where X
    mechanism_state, contact_state = state
    mechanism = mechanism_state.mechanism
    T = cache_eltype(mechanism_state)
    dynamics_result = DynamicsResult{T}(mechanism)
    contact_result = SoftContactResult{T}(mechanism, contact_model)
    closed_loop_dynamics! = let dynamics_result=dynamics_result, contact_result=contact_result
        function (v̇::AbstractArray, ṡ, t, state)
            mechanism_state, contact_state = state
            Contact.contact_dynamics!(contact_result, contact_state, mechanism_state)
            torques = ConstVector(zero(cache_eltype(mechanism_state)), num_velocities(mechanism_state)) 
            dynamics!(dynamics_result, mechanism_state, torques, contact_result.wrenches)
            copyto!(v̇, dynamics_result.v̇)
            copyto!(ṡ, contact_result.ẋ)
#             set_configuration!(mvis, configuration(mechanism_state))
#             yield()
            nothing
        end
    end
    tableau = runge_kutta_4(T)
    storage = ExpandingStorage{T}(state, ceil(Int64, final_time / Δt * 1.001)) # very rough overestimate of number of time steps
    integrator = MuntheKaasIntegrator(state, closed_loop_dynamics!, tableau, storage)
    integrate(integrator, final_time, Δt)
    storage.ts, storage.qs, storage.vs
end

simulate_hack (generic function with 1 method)

In [111]:
copyto!(state, x0)
Contact.reset!(contact_state)
ts, qs, vs = simulate_hack((state, contact_state), 5.0);

In [112]:
setanimation!(mvis, ts, qs)

Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")
Stacktrace:
 [1] try_yieldto(::typeof(Base.ensure_rescheduled), ::Base.RefValue{Task}) at ./event.jl:196
 [2] wait() at ./event.jl:255
 [3] wait(::Condition) at ./event.jl:46
 [4] wait(::Task) at ./task.jl:188
 [5] create_socket(::Dict{Any,Any}) at /home/twan/.julia/packages/WebIO/Rk8wc/src/providers/mux.jl:44
 [6] (::getfield(Mux, Symbol("##5#6")){getfield(Mux, Symbol("##28#29")){Array{SubString{String},1}},typeof(WebIO.create_socket)})(::Function, ::Dict{Any,Any}) at /home/twan/.julia/packages/Mux/FeATY/src/Mux.jl:17
 [7] #1 at /home/twan/.julia/packages/Mux/FeATY/src/Mux.jl:10 [inlined]
 [8] splitquery(::getfield(Mux, Symbol("##1#2")){getfield(Mux, Symbol("##5#6")){getfield(Mux, Symbol("##28#29")){Array{SubString{String},1}},typeof(WebIO.create_socket)},getfield(Mux, Symbol("##1#2")){typeof(Mux.wclose),getfield(Mux, Symbol("##1#2")){getfield(Mux, Symbol("##18#19"

In [ ]:
struct Flatten{}